In [0]:
!wget https://www.dropbox.com/s/4623frt8ufu3uqz/dataset.zip
#!wget https://www.dropbox.com/s/ghqv553yfk5pzja/correction.zip

--2019-01-12 12:10:49--  https://www.dropbox.com/s/4623frt8ufu3uqz/dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4623frt8ufu3uqz/dataset.zip [following]
--2019-01-12 12:10:55--  https://www.dropbox.com/s/raw/4623frt8ufu3uqz/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26bb19edd21fa6cce4ee556fe0.dl.dropboxusercontent.com/cd/0/inline/AZRpUOBVuCeIo2C9dxD_WCtXJy8wFxWcdoH7l8_WeUonILXzNmo6VK5oXVGc1mKT7R-tB_SF7cG1mchdAnEG7qlpimWCV_11xc0UqVOHlFVpEDH80LmZENwM3u21bo2faWL9nsfhhavjvA2Ihguz5MEbBoQHxPe1OkEwa42MdobNCBXpjIlZqWKKi96yk-lThII/file [following]
--2019-01-12 12:10:55--  https://uc26bb19edd21fa6cce4ee556fe0.dl.dropboxusercontent.com/cd/0/inline/AZRpUOBVuCeIo2C9dxD_WCtXJy8wFxWcdoH7l8_WeUonILXzNmo

In [0]:
!mkdir data

In [0]:
%%bash
unzip -q dataset.zip -d data/
#unzip -q correction.zip -d data/

warning [dataset.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [dataset.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #104105).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [0]:
!ls data/

cor	    train_cons.csv  train_vow	   val_cons.csv  val_vow
corr.csv    train_num	    train_vow.csv  val_num	 val_vow.csv
train_cons  train_num.csv   val_cons	   val_num.csv


In [0]:
import tensorflow as tf
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D,BatchNormalization, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import gc
from keras.models import load_model
from google.colab import files

Using TensorFlow backend.


In [0]:
RESIZE_DIM=96

data_dir='data'
#data_dir=os.path.join('..','input')
paths_t_numbers=glob.glob(os.path.join(data_dir,'train_num','*.png'))
paths_t_vowels=glob.glob(os.path.join(data_dir,'train_vow','*.png'))
paths_t_cons=glob.glob(os.path.join(data_dir,'train_cons','*.png'))
paths_cor=glob.glob(os.path.join(data_dir,'cor','*.png'))

paths_train_all=paths_t_numbers+paths_t_vowels+paths_t_cons+paths_cor


paths_v_numbers=glob.glob(os.path.join(data_dir,'val_num','*.png'))
paths_v_vowels=glob.glob(os.path.join(data_dir,'val_vow','*.png'))
paths_v_cons=glob.glob(os.path.join(data_dir,'val_cons','*.png'))

paths_val_all=paths_v_numbers+paths_v_vowels+paths_v_cons


path_label_t_num=os.path.join(data_dir,'train_num.csv')
path_label_t_vow=os.path.join(data_dir,'train_vow.csv')
path_label_t_cons=os.path.join(data_dir,'train_cons.csv')
path_lavel_cor   =os.path.join(data_dir,'corr.csv')

path_label_v_num=os.path.join(data_dir,'val_num.csv')
path_label_v_vow=os.path.join(data_dir,'val_vow.csv')
path_label_v_cons=os.path.join(data_dir,'val_cons.csv')

In [0]:
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(sep=os.sep)[-1]
    return key
  
def get_data(paths_img,path_label=None,resize_dim=None):
   
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img):
        img=cv2.imread(path,cv2.IMREAD_COLOR) # images loaded in color (BGR)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # cnahging colorspace to GRAY
        if resize_dim is not None:
            img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA) # resize image to 28x28
        #X.append(np.expand_dims(img,axis=2)) # expand image to 28x28x1 and append to the list.
        gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
        img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        img = cv2.filter2D(img, -1, kernel)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename') 
        y_label=[df.loc[get_key(path)]['label'] for path in  paths_img] # get the labels corresponding to the images
        y=to_categorical(y_label,60) # transfrom integer value to categorical variable
        return X, y

In [0]:
X_train_n,y_train_n=get_data(paths_t_numbers,path_label_t_num,resize_dim=RESIZE_DIM)
X_train_v,y_train_v=get_data(paths_t_vowels,path_label_t_vow,resize_dim=RESIZE_DIM)
X_train_c,y_train_c=get_data(paths_t_cons,path_label_t_cons,resize_dim=RESIZE_DIM)

X_train_cor,y_train_cor=get_data(paths_cor,path_lavel_cor,resize_dim=RESIZE_DIM)



X_val_n,y_val_n=get_data(paths_v_numbers,path_label_v_num,resize_dim=RESIZE_DIM)
X_val_v,y_val_v=get_data(paths_v_vowels,path_label_v_vow,resize_dim=RESIZE_DIM)
X_val_c,y_val_c=get_data(paths_v_cons,path_label_v_cons,resize_dim=RESIZE_DIM)


X_train_all=np.concatenate((X_train_n,X_train_v,X_train_c,X_train_cor),axis=0)
y_train_all=np.concatenate((y_train_n,y_train_v,y_train_c,y_train_cor),axis=0)

X_train_all.shape, y_train_all.shape
X_show_all=X_train_all
X_show_all.shape


X_val_all=np.concatenate((X_val_n,X_val_v,X_val_c),axis=0)
y_val_all=np.concatenate((y_val_n,y_val_v,y_val_c),axis=0)
X_val_all.shape, y_val_all.shape


X_train_all = X_train_all.reshape(X_train_all.shape[0],96, 96,1).astype('float32')
X_val_all = X_val_all.reshape(X_val_all.shape[0],96, 96,1).astype('float32')

X_train_all.shape
X_val_all.shape

processed 16710/16710
processed 18133/18133
processed 63255/63255
processed 3005/3005
processed 1000/1000
processed 1100/1100
processed 3900/3900


(6000, 96, 96, 1)

In [0]:

def data_aug(X_train,X_test,y_train,y_test,train_batch_size,test_batch_size):
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1.0/255,
        shear_range=0.1,
        #zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    train_batch = train_datagen.flow(X_train,y_train,batch_size=train_batch_size)
    test_batch = test_datagen.flow(X_test,y_test,batch_size=test_batch_size)
    return (train_batch,test_batch)

In [0]:
#X_train_all = X_train_all /255
indices=list(range(len(X_train_all)))
np.random.seed(32)
np.random.shuffle(indices)

train=int(len(indices)*0.80)

# train data
X_train=X_train_all[indices[:train]] 
y_train=y_train_all[indices[:train]]

#test data
X_test=X_train_all[indices[-(len(indices)-train):]] 
y_test=y_train_all[indices[-(len(indices)-train):]]


In [0]:
def my_model(img_size=96,channels=1):
    model = Sequential()
    input_shape = (img_size,img_size,channels)
    
    model.add(Conv2D(64, (3, 3), input_shape=input_shape,activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
   
    
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(512, (3, 3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(Conv2D(512, (3, 3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.000001)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.000001)))
    #model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.000001)))
    #model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(60))
    model.add(Activation('softmax'))
    
    from keras import optimizers
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=sgd)
    return model

model=my_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 64)        640       
_________________________________________________________________
dropout_1 (Dropout)          (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 256)       295168    
__________

In [0]:
batch_size = 256
train_batch, test_batch = data_aug(X_train,X_test,y_train,y_test, batch_size, batch_size)
model=my_model() 
epochs =50
model.fit_generator(train_batch,
                    validation_data=test_batch,
                    epochs=epochs,
                    validation_steps= X_test.shape[0] / batch_size, 
                    steps_per_epoch= X_train.shape[0] / batch_size,
                    verbose=2)

Epoch 1/50
 - 290s - loss: 2.3415 - acc: 0.3748 - val_loss: 1.0782 - val_acc: 0.8254
Epoch 2/50
 - 267s - loss: 0.7543 - acc: 0.7702 - val_loss: 0.4785 - val_acc: 0.9236
Epoch 3/50
 - 268s - loss: 0.4552 - acc: 0.8623 - val_loss: 0.3332 - val_acc: 0.9442
Epoch 4/50
 - 267s - loss: 0.3307 - acc: 0.9015 - val_loss: 0.2966 - val_acc: 0.9593
Epoch 5/50
 - 267s - loss: 0.2687 - acc: 0.9202 - val_loss: 0.2337 - val_acc: 0.9624
Epoch 6/50
 - 267s - loss: 0.2290 - acc: 0.9314 - val_loss: 0.1831 - val_acc: 0.9671
Epoch 7/50
 - 267s - loss: 0.2022 - acc: 0.9402 - val_loss: 0.1818 - val_acc: 0.9709
Epoch 8/50
 - 267s - loss: 0.1816 - acc: 0.9467 - val_loss: 0.1845 - val_acc: 0.9717
Epoch 9/50
 - 267s - loss: 0.1665 - acc: 0.9507 - val_loss: 0.1466 - val_acc: 0.9723
Epoch 10/50
 - 267s - loss: 0.1570 - acc: 0.9544 - val_loss: 0.1671 - val_acc: 0.9749
Epoch 11/50
 - 267s - loss: 0.1414 - acc: 0.9581 - val_loss: 0.1446 - val_acc: 0.9760
Epoch 12/50
 - 267s - loss: 0.1320 - acc: 0.9617 - val_loss: 0.

In [0]:
model.save('512_60.h5')                      

In [0]:
#model=load_model('13_b64_55.h5')
model.evaluate(X_val_all, y_val_all,batch_size=128)

6000/6000 [==============================] - 7s 1ms/step


[0.6778025513986746, 0.9573333336512247]

In [0]:
model.evaluate(X_train_all, y_train_all)

101103/101103 [==============================] - 102s 1ms/step


[0.08689319127750007, 0.9947281485218045]